In [1]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

In [2]:
from web3 import Web3, WebsocketProvider, AsyncHTTPProvider

In [3]:
import json

In [32]:
import pandas as pd
import numpy as np

In [5]:
SCAN_FOLDER_PATH = os.path.join(os.path.dirname(os.getcwd()), "streamlit","scan")

In [12]:
ABI_FOLDER_PATH = os.path.join(os.path.dirname(os.getcwd()), "streamlit", "abi")

In [9]:
sys.path.append(SCAN_FOLDER_PATH)

In [11]:
from scanner import getEventData

In [113]:
load_dotenv(find_dotenv())

True

In [14]:
rpc_url = os.environ.get("RPC_URL")

In [114]:
registry_rpc_url = os.environ.get("REGISTRY_RPC")

In [15]:
wss_rpc = os.environ.get("WSS_RPC")

In [16]:
royaleAddress = os.environ.get("ROYALE_ADDRESS")

In [116]:
registryAddress = os.environ.get("REGISTRY_ADDRESS")

In [17]:
royaleCreateBlockNum = os.environ.get("ROYALE_CREATEBLOCK")

In [18]:
# load abi
with open(os.path.join(ABI_FOLDER_PATH,"RRoyale.json")) as royaleJson:
    royaleAbi = json.loads(royaleJson.read())['abi']

In [109]:
# load registry abi
with open(os.path.join(ABI_FOLDER_PATH,"BurnerAccountRegistry.json")) as registryJson:
    registryAbi = json.loads(registryJson.read())['abi']

In [38]:
#w3 = Web3(Web3.WebsocketProvider(wss_rpc))
w3 = Web3(Web3.HTTPProvider(rpc_url))
#w3 = Web3(asyncProvider)

In [117]:
#w3 = Web3(Web3.WebsocketProvider(wss_rpc))
r3 = Web3(Web3.HTTPProvider(registry_rpc_url))
#w3 = Web3(asyncProvider)

In [39]:
w3.is_connected()

True

In [118]:
r3.is_connected()

True

In [40]:
RoyaleContract = w3.eth.contract(abi=royaleAbi, address=royaleAddress)

In [119]:
RegistryContract = r3.eth.contract(abi=registryAbi, address=registryAddress)

In [15]:
eventName = "GameCreated"

In [20]:
def loadDownloadedData(eventName:str):
    with open(os.path.join(eventName+ ".json")) as data_file:
        datajson = json.loads(data_file.read())
    return pd.DataFrame([event for _, d in datajson.get("blocks").items() for hash, txn in d.items() for i, event in txn.items()])

In [22]:
## get dfs

pKilled = loadDownloadedData("PlayerKilled")
pKilled.head()

,_roomId,_player,timestamp
0,1,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T03:07:32
1,1,0x7c18211dFaB7b8f20dbE5beB79bA7b69FbeFf25b,2023-09-04T03:10:20
2,3,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T03:52:40
3,3,0x7c18211dFaB7b8f20dbE5beB79bA7b69FbeFf25b,2023-09-04T03:52:54
4,4,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T04:14:16


In [23]:
gEnded = loadDownloadedData("GameEnded")
gEnded.head()

,_roomId,_winner,timestamp
0,1,0x9D66Bc3D984d19f8DA431d72979d63488bD38cC2,2023-09-04T03:10:20
1,3,0x9D66Bc3D984d19f8DA431d72979d63488bD38cC2,2023-09-04T03:52:54
2,4,0x9D66Bc3D984d19f8DA431d72979d63488bD38cC2,2023-09-04T04:14:16
3,2,0x1c9CE4d4F48AEaD669d21E89D84BB06f6dc0Fb85,2023-09-04T04:50:12
4,5,0xF2aa741cAfe1E8D2a244A281d0fDE100DB2111fD,2023-09-04T07:09:26


In [53]:
players = pd.DataFrame(np.unique(np.append(pKilled._player.values, gEnded._winner.values)), columns=["burners"] )

In [56]:
players.burners.count()

17

In [123]:
players["burnerParent"] = players.apply(lambda x: getBurnerParent(x["burners"], RegistryContract),axis=1)

In [91]:
players[["totalWins", "totalLosses", "totalGasEarned", "totalGasLost"]] =  players.apply(lambda x: getUserStats(x["burners"], RoyaleContract), axis=1, result_type ='expand')

#players[["totalWins", "totalLosses", "totalGasEarned", "totalGasLost"]] = 

In [99]:
players["totalGames"] = players["totalWins"] + players["totalLosses"]
players["totalRevenue"] = players["totalGasEarned"] + players["totalGasLost"]
players.drop_duplicates(subset="burners", inplace=True)
players.sort_values(by=["totalGames"], ascending=False, inplace=True)
players.reset_index(drop=True, inplace=True)

In [124]:
players.head()

,burners,"(totalWins, totalLosses, totalGasEarned, totalGasLost)",totalWins,totalLosses,totalGasEarned,totalGasLost,totalGames,totalRevenue,burnerParent
0,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,"[0.0, 8.0, 0.0, 210.65]",0.0,8.0,0.000,210.65,8.0,210.650,0x0000000000000000000000000000000000000000
1,0x9D66Bc3D984d19f8DA431d72979d63488bD38cC2,"[3.0, 3.0, 397.794, 0.35]",3.0,3.0,397.794,0.35,6.0,398.144,0x0000000000000000000000000000000000000000
2,0x15F39b053F63797a20ddE324713D890B24a8C690,"[1.0, 5.0, 0.292, 3.35]",1.0,5.0,0.292,3.35,6.0,3.642,0x0000000000000000000000000000000000000000
3,0x02F1f5A58d181cc44c754fF6E246148E6d44F330,"[2.0, 2.0, 1.056, 4.9]",2.0,2.0,1.056,4.90,4.0,5.956,0x0ec5E6656F15851072F67eeaEe50fcF76174CD07
4,0x1a654ae3314CF114dCe3713293BF2D4E5dce2F21,"[2.0, 2.0, 4.802, 1.1]",2.0,2.0,4.802,1.10,4.0,5.902,0x712136E4B6a9fCbC8E7d64DB270376b0023271a4


In [ ]:
players.to_csv()

In [87]:
def getUserStats(address:str, contract:any):
    stats = contract.functions.userStats(address).call()
    stats[2] = stats[2]/1e18
    stats[3] = stats[3]/1e18
    #stats = np.array(stats)
    return stats

In [121]:
def getBurnerParent(address:str, contract:any):
    try:
        return contract.functions.ownerAccounts(address).call()
    except:
        return ""
    return ""

In [122]:
getBurnerParent("0xbbC6C81295e20C33471fceD9f8BDAA069a4cd126", RegistryContract)

'0xA21D9Ab4C71B67782d1E2399C477b613ab1e5b8A'

In [112]:
getUserStats("0xbbC6C81295e20C33471fceD9f8BDAA069a4cd126", RoyaleContract)

[1, 0, 0.096, 0.0]

In [16]:
# get event via logs scrapping (throttle)
getEventData(eventName=eventName, rpc_url=rpc_url, contractAddress=royaleAddress, abi=royaleAbi, contractCreationBlock=int(royaleCreateBlockNum))

State starting from scratch
Scanning event: GameCreated from blocks 186708 - 397079


Current block: 393404 (08-09-2023), blocks in a scan batch: 10000, events processed in a batch 0: : 216520it [02:01, 1775.55it/s]                          

Scanned total 24 'GameCreated', in 121.95424556732178 seconds, total 177 chunk scans performed


In [17]:
# load downloaed data
with open(os.path.join(eventName+ ".json")) as data_file:
    datajson = json.loads(data_file.read())

In [42]:
df = pd.DataFrame([event for _, d in datajson.get("blocks").items() for hash, txn in d.items() for i, event in txn.items()])
df.head()

,_roomId,_creator,timestamp
0,1,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-03T18:18:42
1,2,0x1c9CE4d4F48AEaD669d21E89D84BB06f6dc0Fb85,2023-09-04T02:25:38
2,3,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T03:11:00
3,4,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T04:07:12
4,5,0xF2aa741cAfe1E8D2a244A281d0fDE100DB2111fD,2023-09-04T07:05:26


In [43]:
df.timestamp= pd.to_datetime(df.timestamp)

In [41]:
df["timestamp"]

0     1.693765e+09
1     1.693794e+09
2     1.693797e+09
3     1.693800e+09
4     1.693811e+09
5     1.693844e+09
6     1.693844e+09
7     1.693883e+09
8     1.693884e+09
9     1.693893e+09
10    1.693903e+09
11    1.693908e+09
12    1.693909e+09
13    1.693913e+09
14    1.693923e+09
15    1.693932e+09
16    1.693964e+09
17    1.693964e+09
18    1.694010e+09
19    1.694052e+09
20    1.694069e+09
21    1.694074e+09
22    1.694075e+09
23    1.694086e+09
Name: timestamp, dtype: float64

In [129]:
with open(os.path.join(SCAN_FOLDER_PATH, eventName+ ".json")) as data_file:
    datajson = json.loads(data_file.read())

In [152]:
pd.DataFrame([event for _, d in datajson.get("blocks").items() for hash, txn in d.items() for i, event in txn.items()])

,_roomId,_creator,timestamp
0,1,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-03T18:18:42
1,2,0x1c9CE4d4F48AEaD669d21E89D84BB06f6dc0Fb85,2023-09-04T02:25:38
2,3,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T03:11:00
3,4,0xAB565131e4F17661A71aAE0A8a65ba7B26FD2ccC,2023-09-04T04:07:12
4,5,0xF2aa741cAfe1E8D2a244A281d0fDE100DB2111fD,2023-09-04T07:05:26
5,6,0x7c18211dFaB7b8f20dbE5beB79bA7b69FbeFf25b,2023-09-04T16:12:26
6,7,0x7c18211dFaB7b8f20dbE5beB79bA7b69FbeFf25b,2023-09-04T16:15:58
7,8,0xa98A30aa944Fe80765775aDCf6C9384Af780E134,2023-09-05T03:02:34
8,9,0xa98A30aa944Fe80765775aDCf6C9384Af780E134,2023-09-05T03:16:32
9,10,0x1034A7BE898df18DE2E18D4fb3bAf3E3437044fE,2023-09-05T05:52:32


In [106]:
royaleCreateBlockNum

'186708'

In [ ]:
RoyaleContract.events.GameCreated.create_filter(fromBlock=royaleCreateBlockNum).get_all_entries()